# Test model

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os

# Set up imports
project_root = os.path.abspath("..") 
sys.path.append(project_root)  

# Enable efficient use of GPU memory
from config.gpu.gpu_utils import configure_tensorflow_gpu
configure_tensorflow_gpu()

In [ ]:
import glob
from pathlib import Path
import sklearn
import tensorflow as tf 
import numpy as np
import pandas as pd
from dataloader import make_from_pandas, compute_normalization_stats, test_parser

In [ ]:
# Load the train and validation  sets
train_files = glob.glob('../data/CDL_multiple_scene_ts.parquet/*/*2021*/*.parquet')   # 2021 → train
test_files  = glob.glob('../data/CDL_unique_scene_ts.parquet/*/*2019*/*.parquet')  # 2019 → test

print("Train files:", len(train_files))
print("Test files: ", len(test_files))  

In [ ]:
# Hyperparameters and constants

# Specify constants
date_range = [0, 15, 30, 45, 60, 75, 90, 105, 120, 135, 150, 165, 180, 195, 200]

# Crops we will identify
targeted_cultivated_crops_list = ['Soybeans', 'Rice', 'Corn', 'Cotton']

# Crops we identify as "Cultivated"
other_cultivated_crops_list = [
    'Other Hay/Non Alfalfa', 'Pop or Orn Corn', 'Peanuts', 'Sorghum', 'Oats', 'Peaches',
    'Clover/Wildflowers', 'Pecans', 'Sod/Grass Seed', 'Other Crops', 'Dry Beans', 'Winter Wheat',
    'Alfalfa', 'Potatoes', 'Peas', 'Herbs', 'Rye', 'Cantaloupes', 'Sunflower',
    'Watermelons', 'Sweet Corn', 'Sweet Potatoes'
]

# The label legend
label_legend = ['Uncultivated', 'Cultivated', 'No Crop Growing', 'Soybeans', 'Rice', 'Corn', 'Cotton']

# Define model batch size and time-series bucketing size 
BATCH_SIZE = 1028
DAYS_IN_SERIES = 120
DAYS_PER_BUCKET = 5
MAX_IMAGES_PER_SERIES = (DAYS_IN_SERIES // DAYS_PER_BUCKET) + 1
FRAMES_TO_CHECK = 2
BUCKETING_STRATEGY = "random"
NUM_FEATURES = 16

print("🔢 MAX_IMAGES_PER_SERIES:", MAX_IMAGES_PER_SERIES)
print("📦 Batch shape: [{} x {}]".format(BATCH_SIZE, MAX_IMAGES_PER_SERIES))

In [ ]:
print("📈 Computing normalization parameters...")
means, stds = compute_normalization_stats(
    train_files=train_files,
    label_legend=label_legend,
    targeted_cultivated_crops_list=targeted_cultivated_crops_list,
    other_cultivated_crops_list=other_cultivated_crops_list,
    days_in_series=DAYS_IN_SERIES,
    days_per_bucket=DAYS_PER_BUCKET,
    max_images_per_series=MAX_IMAGES_PER_SERIES,
    frames_to_check=FRAMES_TO_CHECK,
    bucketing_strategy=BUCKETING_STRATEGY,
    batch_size=BATCH_SIZE,
    num_features=NUM_FEATURES
)

In [ ]:
means

In [ ]:
stds

In [ ]:
## Load the trained models

# Model 1: Inception
from models.inception import categorical_focal_loss_var

alpha_vector = [0.25, 0.80, 0.25, 0.25, 0.25, 0.25, 0.25]
alpha_var = tf.Variable(alpha_vector, dtype=tf.float32, trainable=False)
loss_fn = categorical_focal_loss_var(alpha_var, gamma=2.0)

MODEL_PATH = Path('../results/models/inception1d_se_mixup_focal_attention_residual_120days.keras')
MODEL_NAME = 'baseline_inception1d_se_mixup_focal_attention_residual.keras'
FULL_MODEL_PATH = MODEL_PATH / MODEL_NAME

trained_model = tf.keras.models.load_model(str(FULL_MODEL_PATH), custom_objects={'loss': loss_fn})
print(trained_model)

In [ ]:
# Model 2: SimpleCNN 
simplecnn_path = Path('../results/models/simplecnn_120days.keras')
trained_model = tf.keras.models.load_model(
    str(simplecnn_path / 'baseline_simplecnn.keras')
)

In [ ]:
## Load the test data using the test data loader
test_files_ds = make_from_pandas(test_files)

## Predict on the test data (with several points throughout the year)
preds = []
trues = []
lons = []
lats = []
raw_cdl_labels = []
start_days = []
for start_day in date_range:
    test_ds = test_files_ds.map(
        lambda x: test_parser(
            x, norm=True, 
            means=means,
            stds=stds,
            label_legend=label_legend,
            targeted_cultivated_crops_list=targeted_cultivated_crops_list,
            other_cultivated_crops_list=other_cultivated_crops_list,
            days_in_series=DAYS_IN_SERIES,
            days_per_bucket=DAYS_PER_BUCKET,
            max_images_per_series=MAX_IMAGES_PER_SERIES,
            frames_to_check=FRAMES_TO_CHECK,
            bucketing_strategy=BUCKETING_STRATEGY,
            start_day=start_day,
            num_features=NUM_FEATURES
            ),
            num_parallel_calls=tf.data.AUTOTUNE).batch(BATCH_SIZE)
    for X,y,lon,lat,raw_CDL in test_ds:
        pred = tf.argmax(trained_model.predict(X), axis=1)
        true = tf.argmax(y, axis=1)
        preds.append(pred)
        trues.append(true)
        lons.append(lon)
        lats.append(lat)
        raw_cdl_labels.append(raw_CDL)
        start_days.append([start_day] * tf.shape(pred)[0].numpy())

preds = tf.concat(preds, axis=0).numpy()
trues = tf.concat(trues, axis=0).numpy()
lons = tf.concat(lons, axis=0).numpy()
lats = tf.concat(lats, axis=0).numpy()
raw_cdl_labels = tf.concat(raw_cdl_labels, axis=0).numpy()
start_days = list(np.concatenate(start_days, axis=0))
print(preds)
print(trues)

In [ ]:
results = pd.DataFrame({
    'latitude': lats,
    'longitude': lons,
    'Raw CDL Label': raw_cdl_labels,
    'start_day': start_days,
    'true_label': trues,
    'predictions': preds
})

In [ ]:
print(f"Test set accuracy: {sklearn.metrics.accuracy_score(results['true_label'], results['predictions'])}")

In [ ]:
from sklearn.metrics import f1_score

micro_f1_score = f1_score(results['true_label'], results['predictions'], average='micro')
print(f"Micro F1 score: {micro_f1_score}")

macro_f1_score = f1_score(results['true_label'], results['predictions'], average='macro')
weighted_f1_score = f1_score(results['true_label'], results['predictions'], average='weighted')

print(f"Macro F1 score: {macro_f1_score}")
print(f"Weighted F1 score: {weighted_f1_score}")

In [ ]:
# Save Results
results_path = f'../results/test_model_{DAYS_IN_SERIES}days_results.parquet'
results.to_parquet(results_path)

# Analyze model results

In [ ]:
## Read the results table
results = pd.read_parquet(results_path)
results['prediction_day'] = results.start_day + 120
results['year'] = 2019
results['prediction_date'] = pd.to_datetime(results.year * 1000 + results.prediction_day, format='%Y%j') ## Calculate the date
results

In [ ]:
from report_utils import plot_and_save_confusion_matrix

## Display per label accuracy over the whole year
plot_and_save_confusion_matrix(results, label_legend, export_png=False)

In [ ]:
## Prediction accuracy by time of year
from report_utils import plot_and_save_accuracy_by_date

plot_and_save_accuracy_by_date(results, export_png=False)

In [ ]:
from report_utils import plot_crop_classification_comparison

# Example usage:
# 1. Compare worst vs best days
accuracy_by_time_of_year = results.groupby('prediction_date').apply(lambda x: sklearn.metrics.accuracy_score(x['true_label'], x['predictions']))
print(accuracy_by_time_of_year)

worst_start_day = results[results['prediction_date'] == accuracy_by_time_of_year.idxmin()]['start_day'].iloc[0]
best_start_day = results[results['prediction_date'] == accuracy_by_time_of_year.idxmax()]['start_day'].iloc[0]

# Compare worst vs best
fig = plot_crop_classification_comparison(results, worst_start_day, best_start_day, 
                                        "Worst Day", "Best Day")

# 2. Visualize any specific day you want
# For example, if you want to see start_day = 75:
fig = plot_crop_classification_comparison(results, 75, title_1="Day 75")

# 3. Compare any two specific days
# For example, compare start_day 30 vs start_day 120:
fig = plot_crop_classification_comparison(results, 30, 120, "Early Season", "Late Season")

In [ ]:
from report_utils import plot_crop_classification_series

plot_crop_classification_series(results, export_pngs=False, figsize=(24, 8))

In [ ]:
from report_utils import plot_cdl_vs_predictions

# Example usage:
plot_cdl_vs_predictions(results, start_days=[45, 60, 75], export_pngs=False)

In [ ]:
plot_cdl_vs_predictions(results, date_range=date_range, export_pngs=False)

In [ ]:
from report_utils import make_gif_from_pngs, get_results_predictions_dir

import glob
output_dir = get_results_predictions_dir()

# Gif 1
png_paths = sorted(glob.glob(os.path.join(output_dir, "true_vs_pred_*.png")))
make_gif_from_pngs(png_paths, os.path.join(output_dir, "animation_trueVsPred.gif"))

# Gif 2; CDL Vs Predictions
png_paths = sorted(glob.glob(os.path.join(output_dir, "cdl_vs_pred_*.png")))
make_gif_from_pngs(png_paths, os.path.join(output_dir, "animation_cdlLabels.gif"))